In [2]:
!pip install pennylane
import pandas as pd
import numpy as np
import pennylane as qml
import tkinter as tk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, classification_report
from joblib import dump
from PIL import Image, ImageTk
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from tkinter import filedialog, messagebox, simpledialog 

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------  1.7/1.7 MB 104.3 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 104.3 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 11.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/51.0 kB ? eta -:--:--
   -------------------------------- ------- 41.0/51.0 kB ? eta -:--:--
   -------------------------------- ------- 41.0/51.0 kB ? eta -:--:--
   ---------------------------------------- 51.0/51.0 kB 515.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ----------------------- ---------------- 3.2/5.6 MB 68.6 MB/s eta 0:00:01
   --------------------------- ------------ 3.9/5.6 MB 41.1 MB/s eta 0:00:01
   -------------------------------- ------- 4.6/5.6 MB 32.2 MB/s eta 0:00:01
   ------------------------------------ --- 5.2/5.6 MB 27.3 MB/s eta 0:00:01
   --------------------------

In [ ]:
# Initialize global variables
scaler = StandardScaler()
num_qubits = 4
num_layers = 2
learning_rate = 0.1
epochs = 50
dev = qml.device("default.qubit", wires=num_qubits)

# Global variable to hold the PhotoImage object
background_photo = None

# Define quantum circuit and cost function
def circuit(params, x):
    qml.templates.AngleEmbedding(x, wires=range(num_qubits))
    qml.templates.StronglyEntanglingLayers(params, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev)
def qnode(params, x):
    circuit(params, x)
    return qml.expval(qml.PauliZ(0))

def cost(params, X, y):
    predictions = np.array([qnode(params, x) for x in X])
    return np.mean((predictions - y) ** 2)

# Data preprocessing function
def preprocess_data(file_path, target_column):
    global scaler

    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, chunksize=1000)  
    elif file_path.endswith(('.xls', '.xlsx')):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please select a CSV or Excel file.")

    features_list = []
    targets_list = []

    for chunk in df:
        numeric_columns = chunk.select_dtypes(include=[np.number]).columns.tolist()
        selected_features = numeric_columns[:min(len(numeric_columns), num_qubits)] 
        selected_features.remove(target_column) 

        chunk_features = chunk[selected_features].values.tolist()
        chunk_targets = chunk[target_column].values.tolist()
        features_list.extend(chunk_features)
        targets_list.extend(chunk_targets)
    features = np.array(features_list)
    targets = np.array(targets_list)
    features = pd.DataFrame(features, columns=selected_features)
    features.fillna(features.mean(), inplace=True)
    scaled_features = scaler.fit_transform(features)
    selector = SelectKBest(score_func=f_classif, k=min(num_qubits, scaled_features.shape[1]))
    selected_features = selector.fit_transform(scaled_features, targets)

    return selected_features, targets, scaler

# Training and prediction function
def train_and_predict(features, target):
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    params = np.random.randn(num_layers, num_qubits, 3)
    opt = NesterovMomentumOptimizer(learning_rate)
    for epoch in range(epochs):
        params = opt.step(lambda v: cost(v, X_train, y_train), params)

    y_pred = np.array([np.sign(qnode(params, x)) for x in X_test])

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return accuracy, report, params

# File selection and processing function
def on_file_select():
    global background_photo

    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xls *.xlsx")])
    if not file_path:
        return

    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(file_path)
            
        target_column = simpledialog.askstring("Input", f"Available columns:\n{', '.join(df.columns)}\n\nPlease enter the target column:")

        if target_column not in df.columns:
            raise ValueError("Selected target column not found in the file.")

        features, target, scaler = preprocess_data(file_path, target_column)
        accuracy, report, params = train_and_predict(features, target)

        update_results_display(accuracy, report)

        dump(params, 'quantum_model_params.pkl')
        messagebox.showinfo("Success", "Model trained and predictions displayed!")
    except Exception as e:
        messagebox.showerror("Error", f"Error occurred: {str(e)}")

# Function to update results and show background when necessary
def update_results_display(accuracy, report):
    results_text.config(state=tk.NORMAL)
    results_text.delete(1.0, tk.END)
    results_text.insert(tk.END, f"Accuracy: {accuracy}\n\nClassification Report:\n{report}")
    
    # Check if there is content to display
    if accuracy == 0.0 and report.strip() == "":
        results_frame.configure(bg="rgba(255, 255, 255, 0.5)")  # Transparent white background
    else:
        results_frame.configure(bg="white")  # Solid white background
    
    results_text.config(state=tk.DISABLED)

# GUI setup
root = tk.Tk()
root.title("Quantum Machine Learning Model Trainer")
root.geometry("800x600")

# Load and set the background image
try:
    background_image = Image.open(r"C:\Users\Intern13\Desktop\project\testing\test.jpg")  # Replace with the path to your background image
    background_photo = ImageTk.PhotoImage(background_image)
    background_label = tk.Label(root, image=background_photo)
    background_label.place(x=0, y=0, relwidth=1, relheight=1)
except Exception as e:
    print(f"Error loading background image: {e}")

# Create a frame for the header and button
header_frame = tk.Frame(root, bg="#333")
header_frame.pack(fill=tk.X, pady=20)

header_label = tk.Label(header_frame, text="Quantum Machine Learning Model Trainer", fg="white", bg="#333", font=("Helvetica", 16, "bold"))
header_label.pack(pady=10)

select_file_button = tk.Button(header_frame, text="Select CSV or Excel File", command=on_file_select, bg="#4CAF50", fg="white", font=("Helvetica", 12, "bold"))
select_file_button.pack(pady=10)

# Create a frame for the results
results_frame = tk.LabelFrame(root, text="Results", font=("Helvetica", 14, "bold"), bg="white", relief=tk.SOLID, borderwidth=1)  # Solid white background
results_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)

# Use a text widget for results
results_text = tk.Text(results_frame, wrap=tk.WORD, state=tk.DISABLED, font=("Helvetica", 12))
results_text.pack(fill=tk.BOTH, expand=True)

root.mainloop()
